# Overview
We explore making a multi-node blockchain on one machine using multiple threads. The different threads each manage their own blockchain and the kernel shows how they are kept in sync and how tasks like mining a bitcoin works. This code is slightly modified and revised from this [story](https://hackernoon.com/learn-blockchains-by-building-one-117428612f46) by Daniel van Flymen.

In [ ]:
import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4

import requests
from flask import Flask, jsonify, request

# Our Blockchain Class

In [ ]:
class Blockchain(object):
    
    def __init__(self):
        self.current_transactions = []
        self.chain = []
        self.nodes = set()

        # Create the genesis block:
        self.new_block(previous_hash=1, proof=100)

        
    def register_node(self, address):
        '''
        Adds a new node to the list of nodes.

        :param address: <str> Address of the node. Eg. 'http://192.168.0.5:5000'
        :return: None
        '''
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)

        
    def valid_chain(self, chain):
        '''
        Determines if a given blockchain is valid.

        :param chain: <list> A blockchain
        :return: <bool> True if valid, False if not.
        '''
        last_block = chain[0]
        current_index = 1

        while (current_index < len(chain)):
            block = chain[current_index]
            print(f"{last_block}")
            print(f"{block}")
            print("\n-----------\n")
                  
            # [CHECK] Is the hash of the block is incorrect?
            if (block['previous_hash'] != self.hash(last_block)):
                return False

            # [CHECK] Is the Proof of Work is incorrect?
            if (not self.valid_proof(last_block['proof'], block['proof'])):
                return False

            last_block = block
            current_index += 1

        return True

                  
    def resolve_conflicts(self):
        '''
        Our consensus algorithm, which resolves conflicts by replacing our
        chain with the longest one in the network.

        :return: <bool> True if our chain was replaced, False if not.
        '''
        neighbors = self.nodes
        new_chain = None

        # Only looking for chains longer than ours:
        max_length = len(self.chain)

        # Grab and verify the chains from all the nodes in our network:
        for node in neighbors:
            resp = requests.get(f"http://{node}/chain")

            if (resp.status_code == 200):
                length = resp.json()['length']
                chain = resp.json()['chain']

                # [CHECK] Is the length longer and the chain is valid?
                if ((length > max_length) and (self.valid_chain(chain))):
                    max_length = length
                    new_chain = chain

        # Replace our chain if we discovered a new, valid chain longer than ours:
        if new_chain:
            self.chain = new_chain
            return True

        return False

                  
    def new_block(self, proof, previous_hash=None):
        '''
        Creates a new Block in the Blockchain

        :param proof: <int> The proof given by the Proof of Work algorithm.
        :param previous_hash: (Optional) <str> The hash of previous Block.
        :return: <dict> A new Block
        '''
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }

        # Reset the current list of transactions:
        self.current_transactions = []
        self.chain.append(block)
        return block
    

    def new_transaction(self, sender, recipient, amount):
        '''
        Creates a new transaction to go into the next mined Block.

        :param sender: <str> Address of the Sender.
        :param recipient: <str> Address of the Recipient.
        :param amount: <int> Amount
        :return: <int> The index of the Block that will hold this transaction.
        '''
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })
        return self.last_block['index'] + 1
                  

    @property
    def last_block(self):
        return self.chain[-1]
    

    @staticmethod
    def hash(block):
        '''
        Creates a SHA-256 hash of a Block.

        :param block: <dict> Block
        :return: <str>
        '''
        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes.
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

                  
    def proof_of_work(self, last_proof):
        '''
        Simple Proof of Work Algorithm:
        ------------------------------
         - Find a number p' such that hash(pp') contains leading 4 zeroes,
             where p is the previous p'.
         - p is the previous proof, and p' is the new proof.

        :param last_proof: <int>
        :return: <int>
        '''
        proof = 0
        while (self.valid_proof(last_proof, proof) is False):
            proof += 1

        return proof
                  

    @staticmethod
    def valid_proof(last_proof, proof):
        '''
        Validates the Proof

        :param last_proof: <int> The Previous Proof
        :param proof: <int> The Current Proof
        :return: <bool> True if correct, False if not.
        '''
        guess = f"{last_proof}{proof}".encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == '0000'
    

# Encapsulate the Server Logic
We package the blockchain server in a function so that we can make multiple instances of it easily.

In [ ]:
def create_blockchain_process(in_port):

    # Generate a globally unique address for this node:
    node_id = "{}{:04d}".format(str(uuid4()).replace('-', ''), in_port)
    
    # Instantiate the Node:
    app = Flask(node_id)
    
    # Instantiate the Blockchain:
    blockchain = Blockchain()
    
    
    # [ENDPOINT] GET - mine coin
    @app.route('/mine', methods=['GET'])
    def mine():
        # Run the proof of work algorithm to get the next proof:
        last_block = blockchain.last_block
        last_proof = last_block['proof']
        proof = blockchain.proof_of_work(last_proof)

        # We must receive a reward for finding the proof.
        # The sender is '0' to signify that this node has mined a new coin.
        blockchain.new_transaction(
            sender='0',
            recipient=node_id,
            amount=1,
        )

        # Forge the new Block by adding it to the chain:
        block = blockchain.new_block(proof)
        resp = {
            'message': "New Block Forged",
            'index': block['index'],
            'transactions': block['transactions'],
            'proof': block['proof'],
            'previous_hash': block['previous_hash'],
        }
        return jsonify(resp), 200


    # [ENDPOINT] POST - create a new transaction
    @app.route('/transactions/new', methods=['POST'])
    def new_transaction():
        values = request.get_json()

        # [CHECK] Are the required fields in the POST data?
        required = ['sender', 'recipient', 'amount']
        if (not all(k in values for k in required)):
            return 'Missing values', 400

        # Create a new Transaction:
        index = blockchain.new_transaction(
            values['sender'],
            values['recipient'],
            values['amount'],
        )
        resp = {'message': f"Transaction will be added to Block {index}"}
        return jsonify(resp), 201


    # [ENDPOINT] GET - return the full chain
    @app.route('/chain', methods=['GET'])
    def full_chain():
        resp = {
            'chain': blockchain.chain,
            'length': len(blockchain.chain),
        }
        return jsonify(resp), 200


    # [ENDPOINT] POST - register our nodes
    @app.route('/nodes/register', methods=['POST'])
    def register_nodes():
        values = request.get_json()
        nodes = values.get('nodes')
        if (nodes is None):
            return "Error: Please supply a valid list of nodes", 400

        for node in nodes:
            blockchain.register_node(node)

        resp = {
            'message': 'New nodes have been added',
            'total_nodes': list(blockchain.nodes),
        }
        return jsonify(resp), 201


    # [ENDPOINT] GET - resolve nodes
    @app.route('/nodes/resolve', methods=['GET'])
    def consensus():
        replaced = blockchain.resolve_conflicts()

        if replaced:
            resp = {
                'message': 'Our chain was replaced',
                'new_chain': blockchain.chain
            }
        else:
            resp = {
                'message': 'Our chain is authoritative',
                'chain': blockchain.chain
            }

        return jsonify(resp), 200
    
    
    # Run our app!
    app.run(host='0.0.0.0', port=in_port)
    os._exit(0)

# Start a Number of Subprocesses
We use the fork command (only supported on linux and mac) to run multiple instances.

In [ ]:
import os
from time import sleep


out_pids = []
for i in range(5000, 5003):
    cpid = os.fork()
    if not cpid:
        create_blockchain_process(i)
    else:
        out_pids += [cpid]
        
# Make sure everything is started:        
sleep(2)


# Testing
Now that we have a few nodes running we can try out a few standard operations using curl. The operations we have defined are _mine_, _chain_, _transaction/new_ and the commands to handle new nodes. We'll run through a couple of common transactions to show how the nodes work and so that the results can be viewed.

### Mining --
We start off with a very basic mining command.

### Register the Other Nodes --

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5001", "http://localhost:5002"]}' "http://localhost:5000/nodes/register"
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5000", "http://localhost:5002"]}' "http://localhost:5001/nodes/register"
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5000", "http://localhost:5001"]}' "http://localhost:5002/nodes/register"

### Make a Simple Transaction --

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{ "sender": "d4ee26eee15148ee92c6cd394edd974e","recipient": "someone-other-address", "amount": 5}' "http://localhost:5000/transactions/new"

In [ ]:
!curl http://localhost:5000/chain

### Resolve the New Transaction --
We run resolve on each node to update the transaction list.

In [ ]:
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

### Mine a Coin --

In [ ]:
!curl http://localhost:5000/mine

In [ ]:
!curl http://localhost:5000/chain

### Check the Chain --
We check the chain on the other nodes to ensure that it was updated from the new activity.

In [ ]:
!curl http://localhost:5001/mine
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

In [ ]:
!curl http://localhost:5001/chain

In [ ]:
!curl http://localhost:5002/mine
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

In [ ]:
!curl http://localhost:5002/chain

# Clean-Up
Clean everything up so we aren't left with any dangling processes.

In [ ]:
import signal


for cpid in out_pids:
    #os.waitpid(cpid, 0) # to wait for a peaceful exit
    os.kill(cpid, signal.SIGTERM)
    